# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](../images/tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [264]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer
import tensorflow as tf
import keras
from keras import models
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from sklearn import preprocessing
from tensorflow.keras.models import load_model

In [265]:
# your code here
df = pd.read_csv('../data/tic-tac-toe.csv')

In [266]:
df.head()

,TL,TM,TR,ML,MM,MR,BL,BM,BR,class
0,x,x,x,x,o,o,x,o,o,True
1,x,x,x,x,o,o,o,x,o,True
2,x,x,x,x,o,o,o,o,x,True
3,x,x,x,x,o,o,o,b,b,True
4,x,x,x,x,o,o,b,o,b,True


In [267]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 958 entries, 0 to 957
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   TL      958 non-null    object
 1   TM      958 non-null    object
 2   TR      958 non-null    object
 3   ML      958 non-null    object
 4   MM      958 non-null    object
 5   MR      958 non-null    object
 6   BL      958 non-null    object
 7   BM      958 non-null    object
 8   BR      958 non-null    object
 9   class   958 non-null    bool  
dtypes: bool(1), object(9)
memory usage: 68.4+ KB


In [268]:
lb = preprocessing.LabelBinarizer()
df['class'] = lb.fit_transform(df['class'])

In [269]:
for i in df.columns:
    df.replace({i: {'x': 2, 'o': 1, 'b':0}}, inplace = True)

TypeError: Cannot compare types 'ndarray(dtype=int32)' and 'str'

In [278]:
for i in df[:-2].itertuples(index = False):
    df.replace({i:{'b': 0, 'x': 1, 'o': 2}}, regex = True, inplace = True)

In [279]:
df

,TL,TM,TR,ML,MM,MR,BL,BM,BR,class
0,2,2,2,2,1,1,2,1,1,1
1,2,2,2,2,1,1,1,2,1,1
2,2,2,2,2,1,1,1,1,2,1
3,2,2,2,2,1,1,1,0,0,1
4,2,2,2,2,1,1,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...
953,1,2,2,2,1,1,1,2,2,0
954,1,2,1,2,2,1,2,1,2,0
955,1,2,1,2,1,2,2,1,2,0
956,1,2,1,1,2,2,2,1,2,0


In [280]:
y = pd.DataFrame(df['class'])
X = df.drop(columns=['class']).copy()

In [281]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1969)

In [283]:
type(X_train)

pandas.core.frame.DataFrame

## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [326]:
# Build neural network
df_model = models.Sequential()
df_model.add(Dense(units=256, kernel_initializer='uniform', activation='relu', input_dim=9))
df_model.add(Dense(units=128, kernel_initializer='uniform', activation='relu'))
df_model.add(Dense(units=64, kernel_initializer='uniform', activation='relu'))
df_model.add(Dense(activation = 'softmax', units = 2, kernel_initializer = 'uniform'))

# Compile model
df_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train model
df_model.fit(X_train, y_train, batch_size=2, epochs=20)


val_loss, val_acc = df_model.evaluate(X_test, y_test, verbose=0)
print(f'Test loss: {val_loss}')
print(f'Test accuracy: {val_acc}')

df_model.save('tic-tac-toe.model')

Epoch 1/20
766/766 [==============================] - 0s 637us/step - loss: 0.6467 - accuracy: 0.6527
Epoch 2/20
766/766 [==============================] - 0s 506us/step - loss: 0.6139 - accuracy: 0.6723
Epoch 3/20
766/766 [==============================] - 0s 538us/step - loss: 0.5952 - accuracy: 0.7115
Epoch 4/20
766/766 [==============================] - 0s 532us/step - loss: 0.5343 - accuracy: 0.7546
Epoch 5/20
766/766 [==============================] - 0s 509us/step - loss: 0.4941 - accuracy: 0.7768
Epoch 6/20
766/766 [==============================] - 0s 509us/step - loss: 0.4817 - accuracy: 0.7781
Epoch 7/20
766/766 [==============================] - 0s 506us/step - loss: 0.4505 - accuracy: 0.7950
Epoch 8/20
766/766 [==============================] - 0s 505us/step - loss: 0.4131 - accuracy: 0.8159
Epoch 9/20
766/766 [==============================] - 0s 525us/step - loss: 0.3953 - accuracy: 0.8225
Epoch 10/20
766/766 [==============================] - 0s 527us/step - loss: 0.392

## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [327]:
# your code here
model = load_model('tic-tac-toe.model')

y_pred = np.around(model.predict(X_test[10:20]))

for i, j in zip(y_test[10:20].values, y_pred):
    print(f"Actual: {i}, Predicted: {j}")

Actual: [1], Predicted: [0. 1.]
Actual: [1], Predicted: [0. 1.]
Actual: [1], Predicted: [0. 1.]
Actual: [0], Predicted: [1. 0.]
Actual: [1], Predicted: [0. 1.]
Actual: [1], Predicted: [0. 1.]
Actual: [1], Predicted: [0. 1.]
Actual: [1], Predicted: [0. 1.]
Actual: [1], Predicted: [0. 1.]
Actual: [0], Predicted: [1. 0.]


## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

**Which approach(es) did you find helpful to improve your model performance?**

- Increase number of epochs
- Adjust the units in the density layers
- Experiment with adjust the patch size. 
- Tried out Binary_Crossentropy instead of Sparse Categorical Crossentropy